Name : Chhagani Krunal ,ID : 202001158
Name : Ninama Vipul , ID : 202001218
Name : Harsh metkel , ID : 202001154

In [249]:

import random
from qiskit import QuantumCircuit, Aer, execute,assemble,QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, plot_bloch_multivector,plot_bloch_vector
from numpy.random import randint
import numpy as np
import qiskit.quantum_info as qi
from qiskit.quantum_info import random_statevector, Statevector
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator

In [250]:
# function for applying error to all qubits
def apply_error(qc, qubits, p):
    for qubit in qubits:
        if random.random() < p:
            qc.x(qubit)

In [251]:
def bit_flip_circuit():
    # Define the number of qubits
    num_qubits = 1

    # Generate a random statevector

    random_state = random_statevector(2 ** num_qubits)
    # ket 0 state

    zero = Statevector([1,0])

    # Create the circuit
    qr= QuantumRegister(5)
    cr= ClassicalRegister(2)
    qc = QuantumCircuit(qr,cr)  # 5 qubits and 2 classical bits
    qc.initialize(random_state.data,qr[0])
    qc.save_statevector(label='v1')
    # initial = random_state ^ zero ^ zero ^ zero ^ zero  # inital vector is tensor product of 5 qubits

    # Encoding
    qc.cx(0, 1)  # CNOT gate (controlled-X gate) between qubit 0 (main qubit) and qubit 1 (ancillary qubit)
    qc.cx(0, 2)  # CNOT gate between qubit 0 and qubit 2
    qc.save_statevector(label='v2')
    qc.barrier()

    # Apply error
    error_probability = 0.5  # Set the error probability to 0.1
    qubits_to_apply_error = [0, 1, 2]  # Apply error to all three qubits
    apply_error(qc, qubits_to_apply_error, error_probability)  # Introduce errors to the qubits
    qc.save_statevector(label='v3')
    qc.barrier()

    # Error Detection
    qc.cx(0, 3)  # CNOT gate between qubit 0 (ancillary qubit) and qubit 3 (error detection qubit)
    qc.cx(1, 3)  # CNOT gate between qubit 1 and qubit 3 (error detection qubit)
    qc.cx(0,4)   # CNOT gate between qubit 0 and qubit 4 (error detection qubit)
    qc.cx(2,4)   # CNOT gate between qubit 1 and qubit 4 (error detection qubit)
    qc.save_statevector(label='v4')
    qc.measure([3,4],[0,1])  # Measure error detection qubits 3 and 4 and store the result in classical bits 0 and 1

    qc.barrier()
    # Apply error correction based on measurement result
    # if measurement gives 00 then simply ignore
    qc.x(1).c_if(cr,1)  # Apply an X gate to qubit 1 if the measurement result is 1  01 
    qc.x(2).c_if(cr,2) # Apply an X gate to qubit 2 if the measurement result is 2  10
    qc.x(0).c_if(cr,3)  # Apply an X gate to qubit 0 if the measurement result is 3 11
    qc.save_statevector(label='v5')
    qc.barrier()
    # Decoding
    qc.cx(0, 1)  # CNOT gate between qubit 0 and qubit 1
    qc.cx(0,2) # CNOT gate between qubit 0 and qubit 2

    qc.save_statevector(label='v6')
    qc.barrier()
    return qc

In [252]:
# Simulate the circuit
qc = bit_flip_circuit()
qc.draw()

┌────────────────────────────────────────────────┐ v1            v2 »
q7891_0: ┤ Initialize(-0.37322+0.71016j,-0.1197-0.58486j) ├─░────■────■───░──»
         └────────────────────────────────────────────────┘ ░  ┌─┴─┐  │   ░  »
q7891_1: ───────────────────────────────────────────────────░──┤ X ├──┼───░──»
                                                            ░  └───┘┌─┴─┐ ░  »
q7891_2: ───────────────────────────────────────────────────░───────┤ X ├─░──»
                                                            ░       └───┘ ░  »
q7891_3: ───────────────────────────────────────────────────░─────────────░──»
                                                            ░             ░  »
q7891_4: ───────────────────────────────────────────────────░─────────────░──»
                                                            ░             ░  »
c1159: 2/════════════════════════════════════════════════════════════════════»
                                                                             »
«          ░ ┌───┐ v3  ░                      v4        ░                ┌───┐ »
«q7891_0: ─░─┤ X ├─░───░───■─────────■────────░─────────░────────────────┤ X ├─»
«          ░ ├───┤ ░   ░   │         │        ░         ░  ┌───┐         └─╥─┘ »
«q7891_1: ─░─┤ X ├─░───░───┼────■────┼────────░─────────░──┤ X ├───────────╫───»
«          ░ └───┘ ░   ░   │    │    │        ░         ░  └─╥─┘  ┌───┐    ║   »
«q7891_2: ─░───────░───░───┼────┼────┼────■───░─────────░────╫────┤ X ├────╫───»
«          ░       ░   ░ ┌─┴─┐┌─┴─┐  │    │   ░  ┌─┐    ░    ║    └─╥─┘    ║   »
«q7891_3: ─░───────░───░─┤ X ├┤ X ├──┼────┼───░──┤M├────░────╫──────╫──────╫───»
«          ░       ░   ░ └───┘└───┘┌─┴─┐┌─┴─┐ ░  └╥┘┌─┐ ░    ║      ║      ║   »
«q7891_4: ─░───────░───░───────────┤ X ├┤ X ├─░───╫─┤M├─░────╫──────╫──────╫───»
«          ░       ░   ░           └───┘└───┘ ░   ║ └╥┘ ░ ┌──╨──┐┌──╨──┐┌──╨──┐»
«c1159: 2/════════════════════════════════════════╩══╩════╡ 0x1 ╞╡ 0x2 ╞╡ 0x3 ╞»
«                                                 0  1    └─────┘└─────┘└─────┘»
«          v5  ░            v6  ░ 
«q7891_0: ─░───░───■────■───░───░─
«          ░   ░ ┌─┴─┐  │   ░   ░ 
«q7891_1: ─░───░─┤ X ├──┼───░───░─
«          ░   ░ └───┘┌─┴─┐ ░   ░ 
«q7891_2: ─░───░──────┤ X ├─░───░─
«          ░   ░      └───┘ ░   ░ 
«q7891_3: ─░───░────────────░───░─
«          ░   ░            ░   ░ 
«q7891_4: ─░───░────────────░───░─
«          ░   ░            ░   ░ 
«c1159: 2/════════════════════════
«

In [245]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1)
result = job.result()
counts = result.get_counts(qc)
print(counts)
sv1 = result.data(0)['v1']
sv2 = result.data(0)['v2']
sv3 = result.data(0)['v3']
sv4 = result.data(0)['v4']
sv5 = result.data(0)['v5']
sv6 = result.data(0)['v6']

{'00': 1}


In [246]:
sv1.draw('latex', prefix='afterInitializatingTheCircuit:')

<IPython.core.display.Latex object>

In [247]:

sv2.draw('latex', prefix='afterEncoding:')


<IPython.core.display.Latex object>

In [248]:

sv3.draw('latex', prefix='afterErrorIntroduced:')


<IPython.core.display.Latex object>

In [236]:

sv4.draw('latex', prefix='afterErrorDetection:')

<IPython.core.display.Latex object>

In [237]:

sv5.draw('latex', prefix='afterErrorCorrection:')

<IPython.core.display.Latex object>

In [238]:
sv6.draw('latex', prefix='afterDecoding:')

<IPython.core.display.Latex object>

In [239]:
initial = sv1
final = sv6


one = initial.probabilities_dict([0,1,2])
two = final.probabilities_dict([0,1,2])


In [240]:
# sv6.draw('latex', prefix='Statevector6:')
if all(abs(one[qubit] - two[qubit]) < 1e-10 for qubit in one):
    print("Error has been detected and corrected")
else:
    print("Error has not been detected!")

Error has been detected and corrected
